<a href="https://colab.research.google.com/github/karlbehrensg/rna-fashion_mnist/blob/master/Red_Neuronal_Atificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalar las dependencias y configurar el entorno de GPU

In [0]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [2]:
tf.__version__

'2.2.0-rc2'

## Pre procesado de datos

### Cargar el dataset

In [0]:
# Cargar el dataset Fashion Mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

### Normalización de las imágenes

Se divide cada imagen en los conjunto de entrenamiento y de testing entre el valor máximo de cada uno de los píxeles (255).

De este modo, cada píxel se hallará en el rango [0, 1]. Al normalizar las imágenes, nos aseguramos que nuestro modelo de RNA entrenará más rápidamente.


In [0]:
X_train = X_train / 255.0

In [0]:
X_test = X_test / 255.0

### Redimensionar el dataset

Como vamos a utilizar una red neuronal totalmente conectada, vamos a redimensionar los subconjuntos de entrenamiento y testing a formato de vector en lugar de en formato de matriz.

In [0]:
#Como cada imagen tiene 28x28 píxeles, usamos la función reshape en todo el dataset de entrenamiento para convertirlo 
# en vectores de tamaño [-1 (todos los elementos), anchura * altura]
X_train = X_train.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

In [0]:
#Redimensionamos el conjunto de testing del mismo modo
X_test = X_test.reshape(-1, 28*28)

In [9]:
X_test.shape

(10000, 784)

## Paso 4: Construir la Red Neuronal Artificial

### Definir el modelo

Simplemente se define un objeto de modelo Sequential.

In [0]:
model = tf.keras.models.Sequential()

### Añadir la primera capa totalmente conectada (capa Densa)

Hyper-parametros de la capa:
- número de unidades/neuronas: 128
- función de activación: ReLU
- input_shape: (784, )

In [0]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))

### Añadir una capa de Dropout 

Dropout es una técnica de Regularization donde aleatoriamente se asignan a ciertas neuronas de la red el valor cero. De este modo, mientras se entrena, estas neuronas no actualizarán sus valores. Al tener cierto porcentaje de neuronas sin actualizar, el proceso de entrenamiento toma más tiempo pero por contra tenemos menos posibilidades de sufrir overfitting.

In [0]:
model.add(tf.keras.layers.Dropout(0.1))

### Añadir la segunda capa

- unidades: 256
- función de activación: Hard Sigmoid

In [0]:
model.add(tf.keras.layers.Dense(units=256, activation='hard_sigmoid'))

### Añadir una capa de Dropout

In [0]:
model.add(tf.keras.layers.Dropout(0.15))

### Añadir la tercera capa

- unidades: 128
- función de activación: Hard Sigmoid

In [0]:
model.add(tf.keras.layers.Dense(units=128, activation='hard_sigmoid'))

### Añadir una capa de Dropout

In [0]:
model.add(tf.keras.layers.Dropout(0.2))

### Añadir la cuarta capa (capa de salida)

- unidades: número de clases (10 en el caso del Fashion MNIST)
- función de activación: 'softmax' (probabilidades de cada clase)

In [0]:
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Compilar el modelo

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy 

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1

### Entrenar el modelo

In [20]:
model.fit(X_train, y_train, epochs=15)

Epoch 1/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5963 - sparse_categorical_accuracy: 0.7864
Epoch 2/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4028 - sparse_categorical_accuracy: 0.8548
Epoch 3/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3614 - sparse_categorical_accuracy: 0.8686
Epoch 4/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3366 - sparse_categorical_accuracy: 0.8766
Epoch 5/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3201 - sparse_categorical_accuracy: 0.8832
Epoch 6/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3058 - sparse_categorical_accuracy: 0.8879
Epoch 7/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2938 - sparse_categorical_accuracy: 0.8923
Epoch 8/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2865 - sparse_categorical_accuracy: 0.8939
Epoch 9/15
1875/1875 [==========

### Evaluación del modelo y predicción

In [21]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3258 - sparse_categorical_accuracy: 0.8813


In [22]:
print("Test accuracy: {}".format(test_accuracy))

Test accuracy: 0.8812999725341797


## Paso 5 : Guardar el modelo

### Guardar la arquitectura (topoligía) de la red neuronal

In [0]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

### Guardar los pesos de la red neuronal

In [0]:
model.save_weights("fashion_model.h5")